In [24]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
random.seed(9999999)

In [2]:
# Load all of the saved files
train_feat = np.loadtxt('train_feat.gzip', dtype = 'float')
gallery_feat = np.loadtxt('gallery_feat.gzip', dtype = 'float')
query_feat = np.loadtxt('query_feat.gzip', dtype = 'float')
val_feat = np.loadtxt('val_feat.gzip', dtype = 'float')
train_noval_feat = np.loadtxt('train_noval_feat.gzip', dtype = 'float')
train_labels = np.loadtxt('train_labels.gzip', dtype = 'uint16')
gallery_labels = np.loadtxt('gallery_labels.gzip', dtype = 'uint16')
query_labels = np.loadtxt('query_labels.gzip', dtype = 'uint16')
val_labels = np.loadtxt('val_labels.gzip', dtype = 'uint16')
train_noval_labels = np.loadtxt('train_noval_labels.gzip', dtype = 'uint16')
#train_filelist = np.loadtxt('train_filelist.gzip', dtype = 'uint16')
#gallery_filelist = np.loadtxt('gallery_filelist.gzip', dtype = 'uint16')
#query_filelist = np.loadtxt('query_filelist.gzip', dtype = 'uint16')
#val_filelist = np.loadtxt('val_filelist.gzip', dtype = 'uint16')
#train_noval_filelist = np.loadtxt('train_noval_filelist.gzip', dtype = 'uint16')
train_camId = np.loadtxt('train_camId.gzip', dtype = 'uint16') # uint8
gallery_camId = np.loadtxt('gallery_camId.gzip', dtype = 'uint16')
query_camId = np.loadtxt('query_camId.gzip', dtype = 'uint16')
val_camId = np.loadtxt('val_camId.gzip', dtype = 'uint16')
train_noval_camId = np.loadtxt('train_noval_camId.gzip', dtype = 'uint16')

In [5]:
print(train_labels.shape)
print(train_feat.shape)
train_l = np.reshape(train_labels, (7368,1))
print(train_l.shape)


train_matrix = np.append(train_feat, np.reshape(train_labels, (7368,1)), axis=1)



(7368,)
(7368, 2048)
(7368, 1)


In [6]:
def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.

    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = tf.matmul(embeddings, tf.transpose(embeddings))

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = tf.diag_part(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm, 0) - 2.0 * dot_product + tf.expand_dims(square_norm, 1)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.maximum(distances, 0.0)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16

        distances = tf.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances

def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    # Combine the two masks
    mask = tf.logical_and(indices_not_equal, labels_equal)

    return mask


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.logical_not(labels_equal)

    return mask


def _get_triplet_mask(labels):
    """Return a 3D mask where mask[a, p, n] is True iff the triplet (a, p, n) is valid.
    A triplet (i, j, k) is valid if:
        - i, j, k are distinct
        - labels[i] == labels[j] and labels[i] != labels[k]
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    """
    # Check that i, j and k are distinct
    indices_equal = tf.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.logical_not(indices_equal)
    i_not_equal_j = tf.expand_dims(indices_not_equal, 2)
    i_not_equal_k = tf.expand_dims(indices_not_equal, 1)
    j_not_equal_k = tf.expand_dims(indices_not_equal, 0)

    distinct_indices = tf.logical_and(tf.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)


    # Check if labels[i] == labels[j] and labels[i] != labels[k]
    label_equal = tf.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    i_equal_j = tf.expand_dims(label_equal, 2)
    i_equal_k = tf.expand_dims(label_equal, 1)

    valid_labels = tf.logical_and(i_equal_j, tf.logical_not(i_equal_k))

    # Combine the two masks
    mask = tf.logical_and(distinct_indices, valid_labels)

    return mask

In [7]:
def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
    """Build the triplet loss over a batch of embeddings.

    For each anchor, we get the hardest positive and hardest negative to form a triplet.

    Args:
        labels: labels of the batch, of size (batch_size,)
        embeddings: tensor of shape (batch_size, embed_dim)
        margin: margin for triplet loss
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.

    Returns:
        triplet_loss: scalar tensor containing the triplet loss
    """
    # Get the pairwise distance matrix
    pairwise_dist = _pairwise_distances(embeddings, squared=squared)

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    mask_anchor_positive = tf.to_float(mask_anchor_positive)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.reduce_max(anchor_positive_dist, axis=1, keepdims=True)

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.to_float(mask_anchor_negative)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.reduce_min(anchor_negative_dist, axis=1, keepdims=True)

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.reduce_mean(triplet_loss)

    return triplet_loss

In [27]:
n_input = 2048   # input layer (28x28 pixels)
n_hidden1 = 1024 # 1st hidden layer
n_hidden2 = 512 # 2nd hidden layer
n_hidden3 = 256 #3rd hidden layer
n_output = 256   # output layer (0-9 digits)

learning_rate = 1e-4
n_iterations = 10000
batch_person_size = 18
batch_image_size = 4
batch_size = batch_person_size * batch_image_size
dropout = 0.2

X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None])
keep_prob = tf.placeholder(tf.float32) 

layer_1 = tf.layers.dense(X, n_hidden1, tf.nn.relu)
layer_norm_1 = tf.layers.batch_normalization(layer_1)
layer_drop_1 = tf.nn.dropout(layer_norm_1, keep_prob)
layer_2 = tf.layers.dense(layer_drop_1, n_hidden2, tf.nn.relu)
layer_norm_2 = tf.layers.batch_normalization(layer_2)
layer_3 = tf.layers.dense(layer_norm_2, n_hidden3, tf.nn.relu)
layer_drop_3 = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.layers.dense(layer_norm_1, n_output)

hard_triplet_loss = batch_hard_triplet_loss(Y, output_layer, 100)
train_step = tf.train.AdamOptimizer(1e-4).minimize(hard_triplet_loss)

features = output_layer

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

unique_id_train = np.unique(train_labels) #find all unique training IDs

# train on mini batches
for i in range(n_iterations):
    batch_x = np.zeros((batch_size,2048))
    batch_y = np.zeros(batch_size)
    np.random.shuffle(train_matrix)
    unique_id_NN = unique_id_train[np.array(random.sample(range(unique_id_train.shape[0]),batch_person_size))] # pick 18 random unique training IDs, random.sample does not work on numpy array
    for j in range(batch_person_size):
        bool_idx_NN = train_matrix[:,2048] == unique_id_NN[j]
        batch_x[j*batch_image_size:(j+1)*batch_image_size,:] = train_matrix[bool_idx_NN,:2048][:batch_image_size,:]
        batch_y[j*batch_image_size:(j+1)*batch_image_size] = train_matrix[bool_idx_NN,2048][:batch_image_size]
        
        
    
    batch_x, batch_y = train_matrix[:batch_size,:2048],train_matrix[:batch_size,2048]
    sess.run(train_step, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})

    # print loss and accuracy (per minibatch)
    if i%100==0:
        #minibatch_loss, minibatch_accuracy = sess.run([cross_entropy, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:1.0})
        #print("Iteration", str(i), "\t| Loss =", str(minibatch_loss), "\t| Accuracy =", str(minibatch_accuracy))
        print("Iteration", str(i))
        
#test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob:1.0})
#print("\nAccuracy on test set:", test_accuracy)



(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)
(18,)


IndexError: index 18 is out of bounds for axis 0 with size 18

In [ ]:
gallery_feat_Neural = sess.run(features, feed_dict={X: gallery_feat, Y: gallery_labels, keep_prob:1.0})
query_feat_Neural = sess.run(features, feed_dict={X: query_feat, Y: query_labels, keep_prob:1.0})

In [ ]:
feat_dict = {}
feat_dict["Neural Network"] = (query_feat_Neural, gallery_feat_Neural)

print(query_feat_Neural.shape)

In [22]:
#Function that takes as input an index that refers to a query image
# and returns the gallery without the unnecessary pictures in it
# Remove the pictures from the gallery that have the same label and are taken by the same camera as out QUERY[index]
# picture in the query array
def get_removed_gallery (index, t_gallery_feat):
    selected_indices = np.logical_not(np.logical_and(gallery_camId == query_camId[index], gallery_labels == query_labels[index]))
    return t_gallery_feat[selected_indices, :], gallery_labels[selected_indices], gallery_camId[selected_indices]

In [19]:
for key, (t_query_feat, t_gallery_feat) in feat_dict.items():
    # k nearest neighbours implementation
    k = 10
    top_k_accuracies_rank = np.zeros((query_labels.shape[0],k))
    top_k_accuracies_classical = np.zeros((query_labels.shape[0],k))
    top_k_accuracies_recall = np.zeros((query_labels.shape[0],k))

    for i in range (0, query_labels.shape[0], 1):
        gallery_feat_removed, gallery_labels_removed, gallery_camId_removed = get_removed_gallery (i, t_gallery_feat)
        nbrs = NearestNeighbors(n_neighbors=k).fit(gallery_feat_removed)
        distances, indices = nbrs.kneighbors(t_query_feat[i:i+1, :])
        print(gallery_labels_removed[indices], query_labels[i])
        print("camIds:", gallery_camId_removed[indices], query_camId[i])
        is_same_label = (query_labels[i] == gallery_labels_removed[indices])
    
        for j in range (0, k, 1):
            top_k_accuracies_rank[i,j] = np.sum(is_same_label[0, :(j+1)]) != 0
            top_k_accuracies_classical[i,j]=np.sum(is_same_label[0, :(j+1)])/(j+1)
            top_k_accuracies_recall[i,j] = np.sum(is_same_label[0, :(j+1)])/np.sum(gallery_labels_removed==query_labels[i])

    average_precisions = np.zeros((query_labels.shape))
    for j in range (0, query_labels.shape[0], 1):
        recall = top_k_accuracies_recall[j, :]
        precision = top_k_accuracies_classical[j, :]
        index = (recall).argsort()[::-1] # argsort returns the indices that would sort an array (in this case the vector recall)
        recall = recall[index]
        precision = precision[index]
        recall_range = np.arange(0, 1.1, 0.1)
        precision_range = np.zeros((recall_range.shape))
        for i in range (0, recall_range.shape[0], 1):    
            if (precision[recall>=recall_range[i]].size != 0):
                precision_range[i] = np.max(precision[recall>=recall_range[i]])
            else:     
                precision_range[i] = 0
        average_precisions[j] = np.mean(precision_range)
    
    print (key, "Mean average precision:", average_precisions.mean())

[[1368 1368 1368   51 1368 1426 1426 1426 1426 1450]] 3
camIds: [[1 1 1 2 1 2 2 2 2 1]] 1
[[  99  235   51  517   99  235   99 1426  267   99]] 3
camIds: [[2 1 1 1 2 1 1 2 2 2]] 2
[[ 77   6  77 678 678 648  77   6   6   6]] 6
camIds: [[1 2 1 2 2 1 1 2 2 2]] 1
[[ 72   6  72 678   6 678   6  60 816 559]] 6
camIds: [[1 1 1 2 1 2 1 2 1 2]] 2
[[  7   7 382   7   7  22  22 422 441 441]] 7
camIds: [[2 2 1 2 2 1 1 2 1 2]] 1
[[  7   7   7  22 319 319 319  22 319   7]] 7
camIds: [[1 1 1 1 2 2 2 1 2 1]] 2
[[390 584 584  11 721 721  30  30 721 545]] 11
camIds: [[2 2 2 2 2 2 2 2 2 2]] 1
[[ 584  721 1434  721  721  721  584  401  321  401]] 11
camIds: [[2 2 1 2 2 1 1 2 2 2]] 2
[[ 14  14 616  51  51  51  47  47  51   3]] 14
camIds: [[2 2 2 2 2 1 2 1 2 1]] 1
[[ 14  14  14  14  47  47  47  35 291  47]] 14
camIds: [[1 1 1 1 2 1 2 1 1 2]] 2
[[ 17  17  17  17 715  32  32  68 115  32]] 17
camIds: [[2 2 2 2 2 1 1 1 2 1]] 1
[[ 17  17  17  17 715 715 715 715  32 715]] 17
camIds: [[1 1 1 1 2 1 1 1 2 1]] 2
[[  

[[115 115 115 115  17 419  17 473  17  17]] 115
camIds: [[1 1 1 1 1 2 2 2 1 1]] 2
[[725 769 725 725 725 727 725 725 769 725]] 119
camIds: [[1 2 2 1 1 1 1 2 2 2]] 1
[[521 727 174 374 727 174 727 135 303 374]] 119
camIds: [[1 2 2 2 2 2 2 1 1 2]] 2
[[404 650 650 404 650 979 404 699 699 979]] 120
camIds: [[2 2 2 2 2 1 2 2 1 2]] 1
[[646 646 646 509 530 120 646 120 267 120]] 120
camIds: [[1 2 2 1 1 1 2 1 1 1]] 2
[[121 121 558 121 121 558 227 227 578 558]] 121
camIds: [[2 2 1 2 2 1 2 1 2 1]] 1
[[ 121  121  121  829  653  523  829 1370  653  558]] 121
camIds: [[1 1 1 1 2 2 1 1 1 1]] 2
[[123 123 123 123 450 450 136 136 569 134]] 123
camIds: [[2 2 2 2 1 1 1 1 1 1]] 1
[[ 123  123  381  859  859   76 1213  866  450  699]] 123
camIds: [[1 1 2 1 1 1 1 1 1 1]] 2
[[127 127 127 127 494  86  86 494 732 732]] 127
camIds: [[2 2 2 2 1 2 2 1 1 1]] 1
[[127 127 127 127 732 732 732 494 732 494]] 127
camIds: [[1 1 1 1 1 2 2 1 1 1]] 2
[[128 823 823 128 163 552 504 504 552 128]] 128
camIds: [[2 1 1 2 2 1 1 1 1 2]

[[1434  325  316  316  219  328  321 1434  321 1434]] 211
camIds: [[1 2 2 2 2 2 1 1 2 1]] 2
[[212 212 195 212 134 195 660 195 212 195]] 212
camIds: [[2 2 1 2 1 1 2 1 2 2]] 1
[[101 212 212 212 195 762 195 195 212 101]] 212
camIds: [[2 1 1 1 2 2 1 1 1 2]] 2
[[255 213 213 926 858 255 817 755 255 858]] 213
camIds: [[1 2 2 2 2 1 2 2 1 2]] 1
[[751 135 294 294 213 213 294 755 755 862]] 213
camIds: [[1 2 1 1 1 1 1 2 2 1]] 2
[[ 423  423  217  423  217  555  217  506  217 1432]] 214
camIds: [[2 2 1 2 2 2 2 1 2 1]] 1
[[214 214 360 214 557 217 423 555 224 157]] 214
camIds: [[1 1 2 1 2 2 1 2 1 1]] 2
[[ 310  555  555  555  310  442 1371  164  460  217]] 216
camIds: [[1 2 2 2 1 1 2 2 1 1]] 1
[[349 216 424 304 147 557 304 304 304 304]] 216
camIds: [[2 1 2 2 1 1 2 1 2 1]] 2
[[217 217 217 566 224 217 216 566 555 423]] 217
camIds: [[2 2 2 2 1 2 1 2 2 2]] 1
[[217 217 423 557 217 566 566 557 566 164]] 217
camIds: [[1 1 2 2 1 2 1 2 2 2]] 2
[[ 570 1134 1365  218 1365  103  218  491 1365  570]] 218
camIds: [[

[[174 717 304 174 902 647 647 647 304 647]] 306
camIds: [[2 2 1 2 1 1 1 2 1 1]] 1
[[ 752  306  752  306  247  306  306  247  174 1331]] 306
camIds: [[2 1 2 1 2 1 1 2 1 2]] 2
[[586 757 310 586 669 310 310 586 669 745]] 310
camIds: [[1 1 2 1 2 2 2 2 2 1]] 1
[[310 745 757 757 706 310 216 669 745 310]] 310
camIds: [[1 1 1 1 2 1 1 2 1 1]] 2
[[317 748 902 409 505 306 304 505 304 691]] 316
camIds: [[1 1 1 1 1 1 1 1 1 1]] 1
[[763 316 763 211 316 763 316 401 401 401]] 316
camIds: [[1 1 1 2 1 1 1 1 1 2]] 2
[[652 374 325 374 258 521 521 374 763 258]] 317
camIds: [[2 1 1 2 2 1 1 2 2 2]] 1
[[ 652  652  317 1407  317  593  131  174  374 1440]] 317
camIds: [[1 1 1 1 1 1 1 1 2 2]] 2
[[ 319  319  319  319   22  426 1451  426  303  303]] 319
camIds: [[2 2 2 2 1 1 2 1 2 1]] 1
[[319 319 319 303 319  22 303 303 303 303]] 319
camIds: [[1 1 1 2 1 1 2 1 1 2]] 2
[[694 599 599 694 694 599 694 599 694 599]] 320
camIds: [[1 1 2 1 1 2 1 2 2 1]] 1
[[ 660  827 1300  660  802  301  569  599 1300  694]] 320
camIds: [[

[[722 722 401 401 721 722 401 721 401 722]] 401
camIds: [[2 2 2 2 1 1 2 1 2 2]] 1
[[ 384  738 1372   11 1372  721  384  384  721  769]] 401
camIds: [[1 2 1 2 1 1 2 1 1 2]] 2
[[402 695 695 388 388 695 695 388 402 388]] 402
camIds: [[2 2 2 1 1 2 2 1 2 1]] 1
[[402 695 402 695 402 402 136 695 594 593]] 402
camIds: [[1 2 1 2 1 1 1 2 2 2]] 2
[[ 30  30  30 131  30 131 131 636 265 636]] 404
camIds: [[2 2 2 1 2 1 1 2 1 2]] 1
[[   3    3    3  650  834  979   30  650  465 1426]] 404
camIds: [[2 2 2 1 1 2 2 2 1 1]] 2
[[ 33  33 394  33  33 736   6 418 394 499]] 408
camIds: [[2 2 1 2 1 2 1 1 1 1]] 1
[[ 80 435 435 378 435 265 378 242 573 827]] 408
camIds: [[2 1 1 2 1 2 2 1 1 2]] 2
[[390 640 292 292 640 292 640 640 304 390]] 409
camIds: [[2 1 2 2 1 2 1 1 1 2]] 1
[[818 292 304 304 292 292 211 296 325 304]] 409
camIds: [[1 2 1 1 1 1 1 2 1 2]] 2
[[410 673 673 423 569 673 205 410 146 205]] 410
camIds: [[2 1 1 2 2 1 1 2 1 2]] 1
[[410 423 410 410  74 422  74 360 360 426]] 410
camIds: [[1 2 1 1 2 2 2 2 1 1]

...

[[1162  969  969 1162 1162  969  979 1162  969 1007]] 979
camIds: [[1 1 1 1 1 1 2 1 1 2]] 1
[[ 933  932  979  932  932  932 1075  932  932 1075]] 979
camIds: [[1 1 1 1 2 1 2 2 1 2]] 2
[[1077 1077 1273 1077 1077  965  965 1114 1107 1205]] 980
camIds: [[2 2 1 2 2 2 2 1 1 2]] 1
[[ 926  858 1077  980 1054 1008  858  980  926  858]] 980
camIds: [[1 1 1 1 1 1 1 1 1 1]] 2
[[ 982  982  982 1242 1205 1037 1257  965  982  965]] 982
camIds: [[2 2 2 1 1 1 1 1 2 1]] 1
[[ 989  965 1042  965  982 1037  965 1037  982 1042]] 982
camIds: [[2 1 2 1 1 2 1 2 1 1]] 2
[[1155  982  983 1205 1276 1091 1205 1213  965 1064]] 983
camIds: [[1 1 2 1 1 1 2 1 1 1]] 1
[[1002 1127 1127 1205  930 1205  983 1097 1199 1205]] 983
camIds: [[1 1 2 2 2 2 1 1 1 2]] 2
[[ 971  988  971 1122  938 1063  970 1122 1063  988]] 988
camIds: [[1 2 1 1 1 1 1 1 1 2]] 1
[[ 988  971  971 1063 1122  988  988  988  999 1063]] 988
camIds: [[1 1 1 1 1 1 1 1 1 1]] 2
[[ 989 1148 1148 1148  989 1195 1037  968 1145  989]] 989
camIds: [[2 2 2 2 2 1 

[[ 858 1077 1114 1054 1054 1107 1054 1054 1077  980]] 1077
camIds: [[1 1 1 2 1 2 1 2 1 2]] 2
[[ 916  933  933 1195  925 1078  916  991 1078  991]] 1078
camIds: [[2 2 2 1 2 2 2 2 2 2]] 1
[[1078 1078 1078 1179 1032 1032 1032 1142  933 1179]] 1078
camIds: [[1 1 1 2 2 2 2 1 2 2]] 2
[[1032  912  912 1052 1032 1032  912 1052  912 1081]] 1081
camIds: [[1 1 2 1 1 1 2 2 2 2]] 1
[[1081 1081 1081 1081 1227 1159  818  322  818  912]] 1081
camIds: [[1 1 1 1 1 1 1 1 2 2]] 2
[[ 922  922 1026 1088  922  922 1026  970 1407 1088]] 1088
camIds: [[1 2 2 2 2 1 2 1 2 2]] 1
[[1088 1153 1088  922 1133 1153  931 1091 1026 1021]] 1088
camIds: [[1 1 1 2 1 1 1 2 2 1]] 2
[[1008 1099 1008 1008 1008 1008 1008 1008  863 1166]] 1091
camIds: [[2 1 2 2 1 1 1 2 1 1]] 1
[[1049  859 1049 1091  863  859 1199 1049 1091 1049]] 1091
camIds: [[1 2 1 1 2 2 1 1 1 1]] 2
[[1209 1093 1209 1209 1209 1209 1093 1093 1270 1032]] 1093
camIds: [[2 2 2 2 2 1 2 2 2 2]] 1
[[1209 1093 1209 1209 1142 1209 1032 1209 1093 1270]] 1093
camIds: [[2

[[ 841 1181 1181 1181  918 1095 1220 1181 1075 1220]] 1181
camIds: [[1 2 2 2 1 1 1 2 1 1]] 1
[[ 918 1181  918 1181 1004  918 1004 1097 1181 1127]] 1181
camIds: [[1 1 1 1 1 1 1 1 1 1]] 2
[[ 903  655  655 1187  655 1256  932  932  932 1011]] 1182
camIds: [[1 1 1 1 1 2 2 2 1 1]] 1
[[1182 1182 1216 1022 1022 1182 1097 1182  860 1199]] 1182
camIds: [[1 1 2 2 2 1 1 1 2 1]] 2
[[1187 1187 1187 1187 1385 1321  829  829 1370  829]] 1187
camIds: [[2 2 2 2 2 2 1 1 1 2]] 1
[[1187 1187 1187 1187 1385 1182 1013 1013 1013 1022]] 1187
camIds: [[1 1 1 1 2 1 1 1 1 1]] 2
[[ 912  912 1188  912 1188 1188 1032 1270 1209 1209]] 1188
camIds: [[2 2 2 2 2 2 2 2 1 2]] 1
[[ 912  912  912 1052 1188 1062 1052  912  912 1052]] 1188
camIds: [[1 2 2 1 1 2 2 2 1 1]] 2
[[1189 1189 1217 1189 1217 1369  916 1264 1123  925]] 1189
camIds: [[2 2 2 2 1 2 1 1 1 1]] 1
[[ 946  916 1007  946  991 1217 1189  991 1189  991]] 1189
camIds: [[2 1 1 2 2 1 1 1 1 2]] 2
[[1190 1190 1190  929 1331 1313 1091 1190 1313 1068]] 1190
camIds: [[2

[[1097 1153 1075 1097 1097 1097 1178 1017 1153 1017]] 1271
camIds: [[1 2 1 1 2 1 2 2 2 1]] 2
[[1273 1159 1002  892 1063 1205 1276 1273  989 1155]] 1273
camIds: [[2 2 1 2 1 1 1 2 1 1]] 1
[[1107 1273 1114 1114 1114 1114 1273 1107 1114 1114]] 1273
camIds: [[1 1 2 1 1 2 1 2 1 1]] 2
[[ 930 1064 1199 1273 1030 1276 1030 1276 1276 1276]] 1276
camIds: [[1 1 1 1 2 2 2 2 2 2]] 1
[[1276  929  930 1276 1199 1131  929 1030 1127 1030]] 1276
camIds: [[1 2 1 1 1 1 1 2 2 2]] 2
[[1457 1305  822  812 1443  822  404  636 1457  404]] 1285
camIds: [[2 1 1 2 1 1 2 1 2 2]] 1
[[1452  860 1234 1389  860  860 1398 1443 1075  860]] 1285
camIds: [[1 1 1 2 1 2 2 1 2 1]] 2
[[ 518  570  882  570  592  265 1286  265  648  648]] 1286
camIds: [[1 2 2 2 2 2 2 2 1 1]] 1
[[1286  265 1286  425  784  338  748  425  259  784]] 1286
camIds: [[1 1 1 2 2 2 2 2 1 2]] 2
[[1293 1293 1293 1293  802 1300  802 1300  802  802]] 1293
camIds: [[2 2 2 2 2 1 2 2 1 2]] 1
[[1293 1293  802  802  802  802 1300  802  802 1293]] 1293
camIds: [[1

[[ 969  979 1230 1347  887 1234 1385 1406  932  969]] 1390
camIds: [[2 2 1 1 1 1 2 2 1 2]] 2
[[ 838 1371  217  214 1406 1300  566  566  841 1432]] 1392
camIds: [[2 2 1 2 1 2 2 2 2 2]] 1
[[ 343 1445  846 1445 1319 1319  882 1369 1365 1331]] 1392
camIds: [[2 2 1 2 1 1 1 2 2 1]] 2
[[  76 1450 1387  839 1387  839 1387 1387  839  839]] 1397
camIds: [[1 1 1 2 1 2 1 1 2 1]] 1
[[1397 1368 1387 1449 1398 1398 1387 1398 1449 1368]] 1397
camIds: [[1 1 2 1 1 1 2 1 1 1]] 2
[[1449   70  242 1449   70   70  242 1398 1449 1449]] 1398
camIds: [[1 1 1 1 1 2 1 2 2 2]] 1
[[1449 1449 1398 1449 1397 1397 1398  860 1398 1387]] 1398
camIds: [[1 1 1 1 2 2 1 1 1 2]] 2
[[317 805 442 522 805 805 317 805 522 317]] 1405
camIds: [[2 1 2 1 1 1 2 2 1 2]] 1
[[ 790 1251 1307  790  187 1243  187  187 1026  790]] 1405
camIds: [[1 2 2 1 1 2 1 1 1 1]] 2
[[1369 1392 1369 1369 1392  739  739  254  614 1369]] 1406
camIds: [[1 1 1 1 1 1 1 2 2 1]] 1
[[ 614 1162 1112  614 1112  614 1112  882 1387  614]] 1406
camIds: [[1 2 2 1 2 2